<div align="center">
    <h1>DS-210: Programming for Data Science</h1>
    <h1>Lecture 9</h1>
</div>

# 1. Traits
# 2. Collections
# 3. Vectors
# 4. Tests


## Traits

* Common behavior for a set of types
* Some other programming languages: interface

### Sample trait definition

In [30]:
trait Person {
    // method header specifications
    fn get_name(&self) -> String;
    fn get_age(&self) -> u32;
    
    // default implementation of a method 
    fn description(&self) -> String {
        format!("{} ({})",self.get_name(),self.get_age())
    }
}

## Sample trait implementation 1

In [31]:
struct SoccerPlayer {
    name: String,
    age: u32,
    team: String,
}

impl Person for SoccerPlayer {
    fn get_age(&self) -> u32 {
        self.age
    }
    
    fn get_name(&self) -> String {
        self.name.clone()
    }
}

impl SoccerPlayer {
    fn create(name:String,age:u32,team:String) -> SoccerPlayer {
        SoccerPlayer{name,age,team}
    }
}

## Sample trait implementation 2

In [32]:
#[derive(Debug)]
struct RegularPerson {
    year_born: u32,
    first_name: String,
    middle_name: String,
    last_name: String,
}

impl Person for RegularPerson {
    fn get_age(&self) -> u32 {
        2024 - self.year_born
    }
    
    fn get_name(&self) -> String {
        if self.middle_name == "" {
            format!("{} {}",self.first_name,self.last_name)
        } else {
            format!("{} {} {}",self.first_name,self.middle_name,self.last_name)
        }
    }
}

impl RegularPerson {
    fn create(first_name:String,middle_name:String,last_name:String,year_born:u32) -> RegularPerson {
        RegularPerson{first_name,middle_name,last_name,year_born}
    }
}

## Using traits in functions

In [33]:
// sample function accepting object implementing trait
fn long_description(person: &impl Person) {
    println!("{}, who is {} years old", person.get_name(), person.get_age());
}

### Examples

In [35]:
let mlk = RegularPerson::create(
    String::from("Martin"),
    String::from("Luther"),
    String::from("King"),
    1929
);

let zlatan = SoccerPlayer::create(String::from("Zlatan Ibrahimovic"), 40, String::from("AC Milan"));

In [36]:
long_description(&mlk);
long_description(&zlatan);

Martin Luther King, who is 95 years old
Zlatan Ibrahimovic, who is 40 years old


## Using traits in functions: long vs. short form

In [37]:
// short version
fn long_description(person: &impl Person) {
    println!("{}, who is {} old", person.get_name(), person.get_age());
}

// other short version
fn long_description_2(person: &dyn Person) {
    println!("{}, who is {} old", person.get_name(), person.get_age());
}


// longer version
fn long_description_3<T: Person>(person: &T) {
    println!("{}, who is {} old", person.get_name(), person.get_age());
}


In [38]:
long_description(&zlatan);
long_description_2(&zlatan);
long_description_3(&zlatan);

Zlatan Ibrahimovic, who is 40 old
Zlatan Ibrahimovic, who is 40 old
Zlatan Ibrahimovic, who is 40 old


In [39]:
long_description(&mlk);
long_description_2(&mlk);
long_description_3(&mlk);

Martin Luther King, who is 95 old
Martin Luther King, who is 95 old
Martin Luther King, who is 95 old


In [42]:
fn random() -> u32 {
    let mut x = 12236;
    let y = 10329;
    x = x ^ y;
    x = x & y;
    x = x >> 2;
    x = x ^ y;
    return x;
}
fn testme() {
    let mlk = RegularPerson::create(
    String::from("Martin"),
    String::from("Luther"),
    String::from("King"),
    1929
);
    let zlatan = SoccerPlayer::create(String::from("Zlatan Ibrahimovic"), 40, String::from("AC Milan"));
    let someperson: &dyn Person;
    if random() > 5 {
        someperson = &zlatan;
    } else {
        someperson = &mlk;
    }
    // This works because zlatan is a variable of struct type
    long_description(&zlatan);

    // This doesn't work because the actual type of someperson is not known at compile time
    long_description(someperson);
    
    // This works because dyn keyword says it's ok not to know the actual type and to bind the size 
    // of the variable at runtime (more expensive though)
    long_description_2(&zlatan);
    long_description_2(&mlk);
    long_description_2(someperson);
    
    // Doesn't work for same reason the 2nd case
    // long_description_3(someperson);
}
testme();

Error: the size for values of type `dyn Person` cannot be known at compilation time

### So what's up with the different ways to specify traits (It's complicated!!!!)
* `&impl and &T` -> static dispatch (also relevant in the context of return values)
* `&dyn` -> dynamic dispatch (again also relevant in the context of return values)
* `&T` restricts the type especially if you plan to pass multiple arguments of the same type (relevant to inputs)
* Read https://joshleeb.com/posts/rust-traits-and-trait-objects if you want to dig deep but without a background in programming languages and compilers this will not be possible to understand.


## Using traits in functions: multiple traits

In [43]:
use core::fmt::Debug;

fn multiple_1(person: &(impl Person + Debug)) {
    println!("{:?}",person);
    println!("Age: {}",person.get_age());
}

In [44]:
multiple_1(&zlatan);

Error: `SoccerPlayer` doesn't implement `Debug`

In [45]:
multiple_1(&mlk);

RegularPerson { year_born: 1929, first_name: "Martin", middle_name: "Luther", last_name: "King" }
Age: 95


## Using traits in functions: multiple traits

In [46]:
// three options, useful for different settings

// This is good if you want to pass many parameters to the function and the parameters are of different types
fn multiple_1(person: &(impl Person + Debug)) {
    println!("{:?}",person);
    println!("Age: {}",person.get_age());
}

// This is better if you want all your parameters to be of the same type
fn multiple_2<T: Person + Debug>(person: &T) {
    println!("{:?}",person);
    println!("Age: {}",person.get_age());
}

// This is like option 2 but easier to read if your parameter combines many traits
fn multiple_3<T>(person: &T)
    where T: Person + Debug
{
    println!("{:?}",person);
    println!("Age: {}",person.get_age());
}


In [47]:
multiple_1(&mlk);
multiple_2(&mlk);
multiple_3(&mlk);

RegularPerson { year_born: 1929, first_name: "Martin", middle_name: "Luther", last_name: "King" }
Age: 95
RegularPerson { year_born: 1929, first_name: "Martin", middle_name: "Luther", last_name: "King" }
Age: 95
RegularPerson { year_born: 1929, first_name: "Martin", middle_name: "Luther", last_name: "King" }
Age: 95


## Returning types implementing a trait

In [22]:
fn get_zlatan() -> impl Person {
    SoccerPlayer::create(String::from("Zlatan Ibrahimovic"), 40, String::from("AC Milan")) 
}

In [23]:
{
    let zlatan_2 = get_zlatan();
    long_description(&zlatan_2);
};

Zlatan Ibrahimovic, who is 40 old


# <font color="red">2. Collections</font>


## Collections

**Examples:** lists/vectors, hash tables, stack, queue, balanced binary search trees, trees in general

### Why useful

* Storing multiple items, number unknown in advance

* Also the primary reason why generics exist: collections that work for different types of items

### Badly kept secret

* Most tasks: little memory management needed
* Collections will do all the work for you
* Caveat:
  - don't copy large amount of memory
  - use references

### Collection selection
* Driven by efficiency and access needs

# <font color="red">3. Vectors</font>

## Vectors

Extandable and shrinkable array of items:
* Python: list
* C++: vector
* Rust: vector
* Also numpy.resize() but not numpy.reshape().

**Type:** `Vec<T>` stores a collection of values of type `T`

## Creating vectors via macro `vec![...]`

In [8]:
// useful macro: vec![...]
// syntax similar to array

let small_primes = vec![2,3,5,7,11];
small_primes

[2, 3, 5, 7, 11]

In [9]:
// specifc length filled with a given value
let zeros = vec![0;10];
zeros

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [10]:
// size doesn't have to be known in advance
fn get_ones(how_many:usize) -> Vec<i32> {
    vec![1;how_many]
}

let ones = get_ones(13);
ones

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

In [11]:
let a = 13;
let b = 10;
let c = vec![a;b];
c

[13, 13, 13, 13, 13, 13, 13, 13, 13, 13]

## Creating empty via the `new` constructor

In [12]:
// creating a new empty vectors
// with explicit type specification
let v1 : Vec<f64> = Vec::new();
let mut v2 = Vec::<bool>::new();
v1

[]

In [13]:
// won't work: no type specified
let v3 = Vec::new();

Error: Couldn't automatically determine type of variable `v3`.
Please give it an explicit type.

In [14]:
// here Rust can infer what the type is
let mut v4 = Vec::new();
v4.push(123); // <= add element at the end
let c: f32 = 13.0;
v4.push(c);
v4

Error: mismatched types

## Basic operations

In [16]:
// adding elements at the end
println!("{:?} (length={})", v2, v2.len());
v2.push(true);
v2.push(false);
println!("{:?} (length={})", v2, v2.len());

[] (length=0)
[true, false] (length=2)


In [17]:
// accessing specific element
v2[0] = v2[1];
println!("{:?}",v2);

// works because of bool values are copied
// by default

[false, false]


In [20]:
// won't work
struct Seconds(i64);
let x = Seconds(333);
let mut v = Vec::new();
v.push(Seconds(123));
v.push(Seconds(321));
let z = v[0];

Error: cannot move out of index of `Vec<Seconds>`

In [21]:
// references do work
struct Minutes(i64);
{
    let mut v = Vec::new();
    v.push(Minutes(123));
    v.push(Minutes(321));
    let z1 : &Minutes = &v[1];
    let z2 : &mut Minutes = &mut v[1];
};

In [48]:
// reaching out of bounds (panics in debug mode, continues in release mode)
let v = vec![1,2,3,4,5];
v[100]

thread '<unnamed>' panicked at 'index out of bounds: the len is 5 but the index is 100', src/lib.rs:190:40
stack backtrace:
   0: _rust_begin_unwind
   1: core::panicking::panic_fmt
   2: core::panicking::panic_bounds_check
   3: <unknown>
   4: <unknown>
   5: evcxr::runtime::Runtime::run_loop
   6: evcxr::runtime::runtime_hook
   7: evcxr_jupyter::main
note: Some details are omitted, run with `RUST_BACKTRACE=full` for a verbose backtrace.


## Basic operations

In [22]:
// With bound checking
// .get(index) returns Option<&T>
let mut v = vec![0,1,2];
// v[2]
v.get(2)


Some(2)

In [23]:
v.get(100)

None

In [24]:
// removing last element
// returns `Option<T>)
let last = v.push(12);
println!("{:?}, {:?} {:?} {:?}", last, v, v.len(), v.capacity());

(), [0, 1, 2, 12] 4 6


In [25]:
let last = v.push(15);
println!("{:?}, {:?} {:?} {:?}", last, v, v.len(), v.capacity());

(), [0, 1, 2, 12, 15] 5 6


## Iterating over vector
* Iterating: `.iter`
* Mutable iterating: `.iter_mut`

In [27]:
let mut v = vec![2,1,0];
for z in v.iter() { // z in v.iter()
    println!("{}",z);
};

2
1
0


In [28]:
for z in &mut v { // z in v.iter_mut()
    *z += 1;
}
println!("{:?}",v);

[3, 2, 1]


In [29]:
// if you need index as well
for (i,z) in v.iter().enumerate() {
    println!("{}: {}",i,z);
};

0: 3
1: 2
2: 1


In [30]:
// Will not work since vectors have to obey memory ownership semantics (allocated in the heap)
let mut v1 = vec![2,1,0];
let mut v2 = v1;
println!("v1={:?}, v2={:?}", v1, v2);

Error: borrow of moved value: `v1`

In [31]:
let mut v1 = vec![2,1,0];
let mut v2 = v1.clone();
v1[0] = 12;
println!("v1={:?}, v2={:?}", v1, v2);

v1=[12, 1, 0], v2=[2, 1, 0]


In [32]:
fn testme() {
    let mut v1: Vec<u32> = vec![2,1,0];
    let mut v2: &Vec<u32> =  &v1;
    println!("v1={:?}, v2={:?}", v1, v2);
}
testme();

v1=[2, 1, 0], v2=[2, 1, 0]


In [33]:
fn testme() {
    let mut v1: Vec<u32> = vec![2,1,0];
    let mut v2: &mut Vec<u32> =  &mut v1;
    v2[0] = 12;
    // println!("v2={:?}", v2);
    // println!("v1={:?}", v1);
    // But the two below will not work.  Compiler is not smart enough to analyze what is happening inside println
    // So it appears like you are passing two references one that is immutable (v1) and one mutable (v2)
    // which is not safe.
    println!("v2={:?}", v2);
    println!("v1={:?}, v2={:?}", v1, v2);
}
testme();

Error: cannot borrow `v1` as immutable because it is also borrowed as mutable

In [34]:
// Multidimensional vectors
let mut v1: Vec<Vec<u32>> = vec![vec![2,1,0];5];
println!("{:?}", v1);

[[2, 1, 0], [2, 1, 0], [2, 1, 0], [2, 1, 0], [2, 1, 0]]


# <font color="red">4. Tests</font>



* Why are tests useful?
* What is typical test to functional code ratio? (730K lines of code in Meta proxy server, roughly 1:1 ratio of tests to actual code)
    `https://github.com/facebook/proxygen`

In [28]:
fn doubleme(inp: &Vec<f64>) -> Vec<f64> {
    let mut nv = inp.clone();
    for (i, x) in inp.iter().enumerate() {
        nv[i] = *x * 2.0;
    }
    nv
}

#[test]
fn test_doubleme_positive() {
    let v = vec![1.0, 2.0, 3.0];
    let w = doubleme(&v);
    for (x, y) in v.iter().zip(w.iter()) {
        assert_eq!(*y, 2.0 * *x, "Element is not double");
    }
}
#[test]
fn test_doubleme_negative() {
    let v = vec![-1.0, -2.0, -3.0];
    let w = doubleme(&v);
    for (x, y) in v.iter().zip(w.iter()) {
        assert_eq!(*y, 2.0 * *x, "Negative element is not double");
    }
}
#[test]
fn test_doubleme_zero() {
    let v = vec![0.0];
    let w = doubleme(&v);
    for (x, y) in v.iter().zip(w.iter()) {
        assert_eq!(*y, 2.0 * *x, "Zero element is not double");
    }
}
#[test]
fn test_doubleme_empty() {
    let v: Vec<f64> = vec![];
    let w = doubleme(&v);
    assert_eq!(w.len(), 0, "Empty Vector is not empty");
}

fn testme() {
    let v: Vec<f64> = vec![2.0, 3.0, 4.0];
    let w = doubleme(&v);
    println!("V = {:?} W = {:?}", v, w);
}
testme()

V = [2.0, 3.0, 4.0] W = [4.0, 6.0, 8.0]


()

## Read sections 8.1 and 8.3 from the Rust Book.